# <center> Bienes Raices - Mercado Australiano - Prediccion de Precios - Regresion Avanzada

# Tabla de Contenido

1. [Introduccion](#Introduccion)
2. [Configuracion e Instalacion](#Configuracion-e-Instalacion)
3. [Entendimiento de los Datos](#Entendimiento-de-los-Datos)  
4. [Manipulacion y Limpieza de Datos](#Manipulacion-y-Limpieza-de-Datos)
    1. [Dropping Data](#Dropping-Data)
    2. [Derived Data](#Derived-Data)
5. [Analisis de Datos](#Analisis-de-Datos)
    1. [Analisis-Univariable](#Analisis-Univariable)
        1. [Plot Numeric Data](#Plot-Numeric-Data)
        2. [Plot Categorical Data](#Plot-Categorical-Data)
    2. [Analisis-Bivariable](#Analisis-Bivariable)
6. [Preparacion de Datos y Modelado](#Preparacion-de-Datos-y-Modelado)
    1. [Splitting data into Train Test](#Splitting-data-into-Train-Test)
7. [Construccion-del-Modelo](#Model-Building)
    1. [Ridge Regression](#Ridge-Regression)
    2. [Lasso Regression](#Lasso-Regression)
8. [Conclusions y Observaciones](#Conclusiones-y-Observaciones)

# **1.Introduccion**

### Integrates de Grupo:
* Jose Ignacio Rios Villanueva (rios.v.ignacio@gmail.com)
* Saul Enrique Quiroz Castillo (saul85dt@gmail.com)
* Marco Antonio Villarroel Peña (marco.gerenssa@gmail.com)



## Entendimiento del Negocio
Una empresa de vivienda con sede en EE. UU. llamada Surprise Housing ha decidido ingresar al mercado australiano. La empresa utiliza el análisis de datos para comprar casas a un precio inferior a sus valores reales y venderlas a un precio más alto. Con el mismo propósito, la empresa ha recopilado un conjunto de datos de la venta de casas en Australia. Los datos se proporcionan en el archivo CSV a continuación.

La compañía está buscando posibles propiedades para comprar e ingresar al mercado. Debe construir un modelo de regresión utilizando la regularización para predecir el valor real de las posibles propiedades y decidir si invertir en ellas o no.

La empresa quiere saber:

* Qué variables son significativas para predecir el precio de una casa, y
* Qué tan bien esas variables describen el precio de una casa.

Además, determine el valor óptimo de lambda para la regresión de Ridge y Lasso.

### Objetivo del Negocio:

Debe modelar el precio de las casas con las variables independientes disponibles. Luego, la gerencia utilizará este modelo para comprender cómo varían exactamente los precios con las variables. En consecuencia, pueden manipular la estrategia de la empresa y concentrarse en áreas que generarán altos rendimientos. Además, el modelo será una buena manera para que la gerencia entienda la dinámica de precios de un nuevo mercado.

# **2. Configuracion e Instalacion**

In [25]:
!pip install numpy pandas matplotlib plotly scipy scikit-learn statsmodels
!pip install -U nbformat plotly
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import textwrap
from scipy import stats

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet,
    RidgeCV, LassoCV, ElasticNetCV
)
from sklearn.pipeline import Pipeline

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Ignorar advertencias
warnings.filterwarnings("ignore")


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Ignacio\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Ignacio\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


#**3. Entendimiento de los datos**

In [26]:
# Cargar dataset desde Google Drive
url = "https://drive.google.com/uc?id=198LGTVcTgLOm3cqNnLe6dmWuP0XVwfW-"
df_raw = pd.read_csv(url)
print("Shape del dataset:", df_raw.shape)
display(df_raw.head(3))

# Detección heurística de la variable objetivo
target_candidates = [c for c in df_raw.columns if c.lower() in ("saleprice", "sale_price", "price")]
TARGET = target_candidates[0] if target_candidates else df_raw.select_dtypes(include=np.number).columns[-1]
print("Variable objetivo:", TARGET)


Shape del dataset: (1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


Variable objetivo: SalePrice


# **4. Manipulacion y limpieza de datos**

## 4.1. Dropping data

In [27]:
# Copia de trabajo
df = df_raw.copy()

# 4.1 Dropping Data
# Estandarizar valores NA simbólicos
NA_LIKE = {"NA", "Na", "na", "None", "NONE"}
for c in df.select_dtypes(include="object"):
    df[c] = df[c].replace(list(NA_LIKE), np.nan)

# Convertir columnas numéricas mal leídas como texto
for c in df.select_dtypes(include="object"):
    s = df[c].dropna().astype(str).str.replace(",", "").str.strip()
    if len(s) > 0 and (s.str.replace(".", "", 1).str.isdigit().mean() > 0.9):
        df[c] = pd.to_numeric(s, errors="coerce")

# Eliminar columnas con >=80% de valores faltantes
THRESH_DROP = 0.80
cols_high_na = df.columns[df.isna().mean() >= THRESH_DROP].tolist()
df.drop(columns=cols_high_na, inplace=True)
print("Columnas eliminadas por NA alto:", cols_high_na)
print("Shape tras drop columnas:", df.shape)

# Revisar duplicados
dup_count = df.duplicated().sum()
if dup_count > 0:
    df = df.drop_duplicates().reset_index(drop=True)
print("Duplicados eliminados:", dup_count)

Columnas eliminadas por NA alto: ['Alley', 'PoolQC', 'Fence', 'MiscFeature']
Shape tras drop columnas: (1460, 77)
Duplicados eliminados: 0


## 4.2. Derivered data

In [28]:
# 4.2 Derived Data
def derive_features(d: pd.DataFrame) -> pd.DataFrame:
    X = d.copy()
    if set(["FullBath","HalfBath","BsmtFullBath","BsmtHalfBath"]).issubset(X.columns):
        X["TotalBaths"] = (X["FullBath"].fillna(0) + 0.5*X["HalfBath"].fillna(0) +
                            X["BsmtFullBath"].fillna(0) + 0.5*X["BsmtHalfBath"].fillna(0))
    if set(["WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"]).issubset(X.columns):
        X["TotalPorchSF"] = (X["WoodDeckSF"].fillna(0) + X["OpenPorchSF"].fillna(0) +
                             X["EnclosedPorch"].fillna(0) + X["3SsnPorch"].fillna(0) +
                             X["ScreenPorch"].fillna(0))
    if "YearBuilt" in X.columns and "YrSold" in X.columns:
        X["Age"] = X["YrSold"] - X["YearBuilt"]
    if "YearRemodAdd" in X.columns and "YrSold" in X.columns:
        X["RemodAge"] = X["YrSold"] - X["YearRemodAdd"]
    return X

df = derive_features(df)

# **5. Analisis de datos**

## 5.1 Análisis Univariable (target y numéricas)

In [29]:
def plot_distribution(series: pd.Series, title: str):
    fig = px.histogram(series.dropna(), nbins=40, marginal="box", title=title)
    fig.show()

plot_distribution(df[TARGET], f"Distribución de {TARGET}")

skew_target = stats.skew(df[TARGET].dropna())
print("Skew(target) =", round(skew_target, 3))
USE_LOG_TARGET = skew_target > 1.0
if USE_LOG_TARGET:
    df["__y__"] = np.log1p(df[TARGET])
    print("Se usará log1p(TARGET) como y.")
else:
    df["__y__"] = df[TARGET]


Skew(target) = 1.881
Se usará log1p(TARGET) como y.


## 5.2. Análisis Bivariable (correlaciones simples + scatter)

In [30]:
num_cols_for_corr = [c for c in df.select_dtypes(include=np.number).columns if c != "__y__"]
corrs = df[num_cols_for_corr + ["__y__"]].corr(numeric_only=True)["__y__"].sort_values(ascending=False)
print("Top correlaciones con y:")
display(corrs.head(15))
print("Menores correlaciones con y:")
display(corrs.tail(15))


Top correlaciones con y:


__y__           1.000000
SalePrice       0.948374
OverallQual     0.817185
GrLivArea       0.700927
GarageCars      0.680625
TotalBaths      0.673011
GarageArea      0.650888
TotalBsmtSF     0.612134
1stFlrSF        0.596981
FullBath        0.594771
YearBuilt       0.586570
YearRemodAdd    0.565608
GarageYrBlt     0.541073
TotRmsAbvGrd    0.534422
Fireplaces      0.489450
Name: __y__, dtype: float64

Menores correlaciones con y:


PoolArea         0.069798
MoSold           0.057330
3SsnPorch        0.054900
BsmtFinSF2       0.004832
BsmtHalfBath    -0.005149
Id              -0.017942
MiscVal         -0.020021
OverallCond     -0.036868
YrSold          -0.037263
LowQualFinSF    -0.037963
MSSubClass      -0.073959
KitchenAbvGr    -0.147548
EnclosedPorch   -0.149050
RemodAge        -0.568136
Age             -0.587290
Name: __y__, dtype: float64

In [31]:
for col in ["GrLivArea", "GarageCars", "TotalBsmtSF", "OverallQual"]:
    if col in df.columns:
        fig = px.scatter(df, x=col, y="__y__", trendline="ols", title=f"{col} vs y")
        fig.show()

## 5.3. Análisis Multivariable (heatmap de correlación numérica)

In [32]:
corr_matrix = df.select_dtypes(include=np.number).corr(numeric_only=True)
fig = px.imshow(corr_matrix, title="Matriz de correlación (numéricas)", aspect="auto")
fig.show()

En el análisis de correlaciones, se observa que las variables “GrLivArea” (área habitable), “GarageCars”, “TotalBsmtSF” y “OverallQual” (calidad general de la vivienda) presentan correlaciones positivas altas con el precio de venta, lo que indica que, a innmuebles que tienen mayor área construida, mejor calidad en cuanto a instalaciones y acabados o más espacio de garaje, corresponde un valor de venta elevado.  

En contraste, las variables relacionadas con la antigüedad, como “Age” y “RemodAge”, muestran correlaciones negativas moderadas con el precio de venta, esto es muy lógico, puesto que, las viviendas o inmuebles que son más antiguas suelen presentar precios de venta más bajos (el signo negativo revela una relación inversa con el precio de venta).

Sin embargo, las correlaciones solamente muestran el comportamiento de las variables respecto al precio de venta, aunque estos primeros hallazgos son valiosos, no revelan una relación de causalidad entre variables explicativas y la variable dependiente, en este caso, el precio de venta. Para entender cómo inciden cada una de las variables independientes en el precio de venta, a continuación construimos el modelo de regresión.  

# **6.Preparación de datos y Modelado**

In [33]:
FEATURES = [c for c in df.columns if c not in [TARGET, "__y__"]]
X = df[FEATURES].copy()
y = df["__y__"].copy()

numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = [c for c in X.columns if c not in numeric_features]

# Imputación
num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")
preprocess = ColumnTransformer(transformers=[
    ("num", num_imputer, numeric_features),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_enc = preprocess.fit_transform(X_train)
X_test_enc  = preprocess.transform(X_test)


# **7. Construcción del modelo**

In [34]:
# RFE para selección de features
scaler_rfe = StandardScaler()
X_train_scaled = scaler_rfe.fit_transform(X_train_enc)
lr = LinearRegression()
rfe = RFE(estimator=lr, n_features_to_select=min(50, X_train_scaled.shape[1]))
rfe.fit(X_train_scaled, y_train)
selected_features_mask = rfe.support_
feature_names = list(numeric_features) + list(preprocess.transformers_[1][1].get_feature_names_out(categorical_features))
selected_final = [f for f, s in zip(feature_names, selected_features_mask) if s]

In [35]:

# RidgeCV / LassoCV / ElasticNetCV usa CV interno
alphas_ridge = np.logspace(-3, 3, 25)
alphas_lasso = np.logspace(-3, 1.5, 25)
alphas_enet  = np.logspace(-3, 1.5, 25)
l1_ratios_enet = np.linspace(0.05, 0.95, 10)

# Submatrices finales
Xtr_sub = X_train_enc[:, selected_features_mask]
Xte_sub = X_test_enc[:, selected_features_mask]

# Ridge
ridge_cv = RidgeCV(alphas=alphas_ridge)
ridge_cv.fit(Xtr_sub, y_train)
y_pred_ridge = ridge_cv.predict(Xte_sub)
r2_ridge = r2_score(y_test, y_pred_ridge)
rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))

# Lasso
lasso_cv = LassoCV(alphas=alphas_lasso, max_iter=5000, cv=5, random_state=42)
lasso_cv.fit(Xtr_sub, y_train)
y_pred_lasso = lasso_cv.predict(Xte_sub)
r2_lasso = r2_score(y_test, y_pred_lasso)
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))

# ElasticNet
enet_cv = ElasticNetCV(alphas=alphas_enet, l1_ratio=l1_ratios_enet, max_iter=5000, cv=5, random_state=42)
enet_cv.fit(Xtr_sub, y_train)
y_pred_enet = enet_cv.predict(Xte_sub)
r2_enet = r2_score(y_test, y_pred_enet)
rmse_enet = np.sqrt(mean_squared_error(y_test, y_pred_enet))

# Selección del mejor modelo
scores = {"Ridge": r2_ridge, "Lasso": r2_lasso, "ElasticNet": r2_enet}
best_model_name = max(scores, key=lambda k: scores[k])
best_pred = {"Ridge": y_pred_ridge, "Lasso": y_pred_lasso, "ElasticNet": y_pred_enet}[best_model_name]

print(f"Mejor modelo: {best_model_name}")

Mejor modelo: Ridge


En esta porción de codigo se usa mean_squared_error con el parámetro squared=False para calcular el RMSE (Root Mean Squared Error), que representa la raíz cuadrada del error cuadrático medio. A diferencia del MSE, el RMSE mantiene las mismas unidades que la variable objetivo y facilita la interpretación del desempeño del modelo, ya que indica el promedio de desviación de las predicciones respecto a los valores reales en la misma escala del target.

In [36]:
scores = {
    "Ridge": (r2_ridge, rmse_ridge),
    "Lasso": (r2_lasso, rmse_lasso),
    "ElasticNet": (r2_enet, rmse_enet)
}
best_model_name = max(scores, key=lambda k: scores[k][0])
print("Mejor modelo:", best_model_name, "->", scores[best_model_name])

best_pred = {"Ridge": y_pred_ridge, "Lasso": y_pred_lasso, "ElasticNet": y_pred_enet}[best_model_name]
residuals = y_test - best_pred

fig = px.histogram(residuals, nbins=40, title=f"Residuos - {best_model_name}")
fig.show()

fig = px.scatter(x=best_pred, y=residuals, labels={"x":"Predicción", "y":"Residuo"},
                 title=f"Residuos vs Predicción - {best_model_name}")
fig.add_hline(y=0)
fig.show()


Mejor modelo: Ridge -> (0.9075268225941574, np.float64(0.1313640401847977))


## 7) Importancia / Coeficientes (interpretación)

In [37]:
# Función robusta para extraer coeficientes y devolver Series indexada por nombres
def coef_importances_any(estimator, feature_names):
    """
    Acepta:
      - estimator: puede ser un Pipeline (con named_steps['model']) o un estimador ajustado (RidgeCV, LassoCV, etc.)
      - feature_names: lista con los nombres de las features (orden debe coincidir con X usado en fit)
    Devuelve:
      - pd.Series con coeficientes, index=feature_names, ordenada por |coef| desc.
    """
    # Si es pipeline, extraer el paso 'model'
    if hasattr(estimator, "named_steps"):
        model = estimator.named_steps.get("model")
    else:
        model = estimator

    # Asegurar que el modelo tenga coef_
    if not hasattr(model, "coef_"):
        raise ValueError("El estimador no tiene atributo coef_. ¿No está entrenado o no es lineal?")

    coefs = np.array(model.coef_).ravel()  # asegurar forma 1D

    # Asegurar que el número de coeficientes coincide con el número de nombres de características
    if len(coefs) != len(feature_names):
         raise ValueError("El número de coeficientes no coincide con el número de nombres de características proporcionados.")

    # Si hay intercepto y quieres incluirlo, se podría agregar aquí (opcional)
    return pd.Series(coefs, index=feature_names).sort_values(key=lambda x: x.abs(), ascending=False)

# Mapeo correcto: usar los objetos ajustados (RidgeCV, LassoCV, ElasticNetCV)
estimators_map = {
    "Ridge": ridge_cv,
    "Lasso": lasso_cv,
    "ElasticNet": enet_cv
}

best_estimator_obj = estimators_map[best_model_name]

# Obtener la Series de coeficientes mapeada a las features seleccionadas
coefs = coef_importances_any(best_estimator_obj, selected_final)

coefs_sorted = coefs.sort_values(ascending=False)

# Mostrar las 25 variables con coeficientes más grandes (positivos)
print("Top 25 coeficientes más grandes (positivos):")
display(coefs_sorted.head(25))

print("Top 15 coeficientes más pequeños (negativos):")
display(coefs_sorted.tail(15))

Top 25 coeficientes más grandes (positivos):


Neighborhood_StoneBr    0.139779
Neighborhood_Crawfor    0.119393
SaleCondition_Alloca    0.109934
Exterior1st_BrkFace     0.106339
BldgType_2fmCon         0.103310
SaleType_ConLD          0.094172
SaleType_New            0.084916
Neighborhood_NridgHt    0.084241
Functional_Typ          0.071772
BsmtExposure_Gd         0.065371
KitchenQual_Ex          0.057483
OverallQual             0.057375
GarageCars              0.056844
LotConfig_CulDSac       0.055960
Condition1_Norm         0.051308
SaleCondition_Normal    0.050659
Exterior1st_MetalSd     0.044321
BsmtQual_Ex             0.040716
TotalBaths              0.037973
OverallCond             0.035895
BsmtFullBath            0.030560
HeatingQC_Ex            0.030298
GarageType_Attchd       0.029754
Fireplaces              0.029305
ExterCond_TA            0.027809
dtype: float64

Top 15 coeficientes más pequeños (negativos):


HouseStyle_1Story      -0.013287
Foundation_CBlock      -0.020494
Exterior2nd_MetalSd    -0.024321
MSZoning_RL            -0.035256
KitchenAbvGr           -0.052770
BsmtCond_Fa            -0.058720
CentralAir_N           -0.059442
Neighborhood_Edwards   -0.062004
Neighborhood_MeadowV   -0.077861
MSZoning_RM            -0.091746
Heating_Grav           -0.146224
Functional_Maj2        -0.155869
MSZoning_C (all)       -0.180268
Condition2_PosN        -0.356788
RoofMatl_ClyTile       -0.699057
dtype: float64

In [38]:
# Gráfico con plotly
fig = px.bar(coefs_sorted.head(25).reset_index().rename(columns={"index": "feature", 0: "coef", 1: "value"}),
             x=coefs_sorted.head(25).index, y=coefs_sorted.head(25).values,
             title=f"Top 25 coeficientes por |peso| - {best_model_name}")
# alternativo: construir DataFrame explícito para evitar problemas de nombres
top25 = coefs_sorted.head(25).reset_index()
top25.columns = ["feature", "coef"]
fig = px.bar(top25, x="feature", y="coef", title=f"Top 25 mayores coeficientes por |peso| - {best_model_name}")
fig.update_layout(xaxis_tickangle=-45, xaxis_tickfont_size=10)
fig.show()

In [39]:
# Gráfico con plotly
fig = px.bar(coefs_sorted.tail(15).reset_index().rename(columns={"index": "feature", 0: "coef", 1: "value"}),
             x=coefs_sorted.tail(15).index, y=coefs_sorted.tail(15).values,
             title=f"Top 15 coeficientes más pequeños por |peso| - {best_model_name}")
# alternativo: construir DataFrame explícito para evitar problemas de nombres
tail15 = coefs_sorted.tail(15).reset_index()
tail15.columns = ["feature", "coef"]
fig = px.bar(tail15, x="feature", y="coef", title=f"Top 15 coeficientes más pequeños por |peso| - {best_model_name}")
fig.update_layout(xaxis_tickangle=-45, xaxis_tickfont_size=10)
fig.show()

## 8) Predicción

In [40]:
y_test_pred = best_pred
if USE_LOG_TARGET:
    y_test_true  = np.expm1(y_test)
    y_test_predR = np.expm1(y_test_pred)
else:
    y_test_true  = y_test
    y_test_predR = y_test_pred

r2_final   = r2_score(y_test_true, y_test_predR)
rmse_final = mean_squared_error(y_test_true, y_test_predR)
print(f"[Final] {best_model_name} -> R2: {r2_final:.4f} | RMSE: {rmse_final:.4f}")

pred_df = pd.DataFrame({"y_true": y_test_true, "y_pred": y_test_predR})
display(pred_df.head(10))


[Final] Ridge -> R2: 0.9261 | RMSE: 566782414.2128


,y_true,y_pred
892,154500.0,151170.122532
1105,325000.0,309146.771141
413,115000.0,105323.494055
522,159000.0,148165.240493
1036,315500.0,322314.083543
614,75500.0,80770.277384
218,311500.0,242946.269580
1160,146000.0,144355.674083
649,84500.0,77679.992471
887,135500.0,138054.120710


# **8.Conclusiones**

In [41]:
def print_conclusions():
    txt = f"""
### Resultados finales

R2 Scores:
* Ridge:      {r2_ridge:.4f}
* Lasso:      {r2_lasso:.4f}
* ElasticNet: {r2_enet:.4f}

Mejor modelo: {best_model_name}

Parámetros óptimos:
* Ridge alpha: {ridge_cv.alpha_:.6f}
* Lasso alpha: {lasso_cv.alpha_:.6f}
* ElasticNet alpha: {enet_cv.alpha_:.6f}, l1_ratio: {enet_cv.l1_ratio_:.2f}
"""
    print(textwrap.dedent(txt))

print_conclusions()


### Resultados finales

R2 Scores:
* Ridge:      0.9075
* Lasso:      0.9005
* ElasticNet: 0.9074

Mejor modelo: Ridge

Parámetros óptimos:
* Ridge alpha: 1.778279
* Lasso alpha: 0.001000
* ElasticNet alpha: 0.001540, l1_ratio: 0.05



## Conclusiones tecnicas
El análisis de los resultados permite concluir que el modelo de Regresión Ridge demostró un rendimiento predictivo ligeramente superior en el conjunto de prueba, con un coeficiente de determinación (R²) de 0.9075. Este valor supera marginalmente a los obtenidos por los modelos Lasso (R² = 0.9005) y ElasticNet (R² = 0.9074). La evidencia sugiere que la estrategia de penalización L2 de Ridge consigue un equilibrio más efectivo entre el sesgo y la varianza, al regular los coeficientes de manera uniforme sin llegar a eliminar variables. Esta característica resulta ventajosa para preservar la información contenida en los predictores, mitigando el sobreajuste sin comprometer la capacidad explicativa del modelo.

En contraste, el modelo Lasso, con su penalización L1, tiende a producir coeficientes exactamente iguales a cero, lo que conduce a modelos más simples e interpretables. Sin embargo, para este conjunto de datos en particular, esta selectividad implicó una reducción marginal en su poder predictivo. Por su parte, el modelo ElasticNet, que combina las penalizaciones L1 y L2, no logró superar el desempeño de Ridge, lo que indica que la incorporación de la capacidad de selección de variables de Lasso no aportó una ventaja significativa en esta aplicación.

La robustez del modelo Ridge se ve respaldada por el valor de su hiperparámetro de regularización (α ≈ 1.78), que indica un nivel moderado de penalización. Este resultado refuerza la estabilidad del modelo frente a la multicolinealidad presente entre las variables predictoras más influyentes, como el área habitable, la capacidad del garaje, la superficie total del sótano y la calidad general de la vivienda.

Finalmente, se observa una ligera discrepancia entre las métricas evaluadas en la escala transformada y la original. Esta diferencia es atribuible a la transformación logarítmica aplicada a la variable objetivo (SalePrice) para estabilizar la varianza y manejar su sesgo positivo. Mientras que en la escala logarítmica los errores relativos se distribuyen de manera más estable, la reversión a la escala original mediante la función exponencial amplifica los errores absolutos, lo que explica la reducción natural en el valor final de R². Este comportamiento es esperable y confirma la importancia de considerar la métrica en la escala de interés comercial para una evaluación realista del modelo.

## Insights

El coeficiente R² = 0.91 indica un excelente nivel de ajuste de la regresión, es decir, de los valores predichos respecto a los datos observados, por lo tanto, este modelo de regresión se puede emplear con mucha confianza para realizar operaciones en el mercado inmobiliario de Australia, tanto a favor de clientes naturales (familias o individuos que desean comprar o vender viviendas) como de clientes empresariales (empresas inmobiliarias y constructoras) y también para facilitar la toma de decisiones de política pública (relativa a régimen impositivo inmobiliario, urbanismo, revitalización urbana, mejoramiento de barrios, etc.).

Un primer hallazgo importante de este ejercicio, es que el valor de venta de los bienes inmuebles está fuertemente influenciado, de manera negativa, por el material de los techos, específicamente los techos de arcilla o teja (RoofMatl_ClyTile); este aspecto revela una oportunidad de negocio interesante, especialmente para actores empresariales (privados o público-privados), puesto que podrían adquirir viviendas con techo de teja o arcilla a precios bajos, remodelarlas cambiando el material de la cubierta y techos, y reingresarlas al mercado inmobiliario a precios mucho más competitivos, recuperando la inversión y obteniendo ganancias, además de dinamizar el mercado inmobiliario.  

Otro hallazgo interesante, es que la cercanía de los bienes inmuebles a parques o áreas verdes es un factor que le suma valor a las propiedades (Condition2_PosN), puesto que el signo negativo quiere decir que mientras menor sea la distanca de una vivienda a un parque o área verde, mayor es su precio de venta (relación inversa). En este sentido, la recomendación a nivel de política pública sería aumentar las áreas verdes urbanas como una estrategia para incrementar el valor de la propiedad inmobiliaria.  

Por otro lado, variables del tipo Neighborhood revelan que la localizacióon en una u otra urbanización, incide en el valor de venta, en algunos casos de manera positiva y en otros de manera negativa, por lo tanto, la recomendación sería invertir en propiedades localizadas en las urbanizaciones de Stone Brook, Crawford y Northridge Heights, donde la localización es un factor que suma valor a los bienes inmuebles, y evitar invertir en las urbanizaciones de Edwards y Meadow Village, puesto que en éstas últimas su localización es un aspecto que las devalúa.  

Otros aspectos menores, serían: (i) que el ladrillo expuesto en la fachada de las viviendas (Exterior1st_BrkFace) es un aspecto que le suma valor al precio de venta (muy probablemente tenga que ver con algún tipo de acabado, como revestimiento en forma de ladrillo o empleo de ladrillo "gambote", y no se trata de ladrillo expuesto por obra inconclusa); (ii) las viviendas que inicialmente fueron diseñadas como unifamiliares y que posteriormente fueron divididas (BldgType_2fmCon), así como viviendas vinculadas pero con escrituras separadas, por ejemplo en condominios (SaleCondition_Alloca), también están valoradas de manera positiva en el mercado australiano; finalmente (iii) viviendas nuevas (SaleType_New) son positivamente valoradas en el mercado inmobiliario.


